# SOLID Principles
1. The Single-Responsibility Principle **(SRP)**
2. The Open-Closed Principle **(OCP)**
3. The Liskov Substitution Principle **(LSP)**
4. The Interface Segregation Principle **(ISP)**
5. The Dependency Inversion Principle **(DIP)**

These are not to be applied in specific order but a collection of principles gathered as best practices, each has an acronym, other popular best practices like DRY (Don't repeat yourself) and KISS (Keep It Small and Simple - Stupid Simple).

## THE SINGLE-RESPONSIBILITY PRINCIPLE (SRP)
### A class should have one, and only one, reason to change...
In other words, every component of your code (in general a class (?) but also a function) should have one and only one responibility. As a consequence of that, there should be only a reason to change it.

In [2]:
# BAD APPROACH: a single function takes care of an entire operation
import numpy as np
def math_operations(list_):
    # Compute average
    print(f"The mean is: {np.mean(list_)}")

    # Compute max
    print(f"The mean is: {np.max(list_)}")

math_operations(list_ = [1, 2, 3, 4])

The mean is: 2.5
The mean is: 4


In [4]:
# GOOD APPROACH: main functions coordinates how every atomic function is to be executed
# therefore, is the only one susceptible to changes
def get_mean(list_):
    """Compute Mean"""
    print(f"The mean is: {np.mean(list_)}")


def get_max(list_):
    """Compute Mean"""
    print(f"The max is: {np.max(list_)}")

def main(list_):
    # first: compute averate
    get_mean(list_)

    # second: compute max
    get_max(list_)

main([1, 2, 3, 4])

The mean is: 2.5
The max is: 4


## THE OPEN-CLOSED PRINCIPLE (OCP)
### Software entities... should be open for exentsion but closed for modification
In Python this can be applied using Abstract Classes to blueprint how every inheriting class should work, that way if we need to add a new operation we just have to add a new subclass inheriting from Operations but the base class is not modified.

In [7]:
import numpy as np
from abc import ABC, abstractmethod

class Operations(ABC):
    """Operations"""
    @abstractmethod
    def operation():
        pass

class Mean(Operations):
    """Compute mean"""
    # overwrite the abstract class operation method
    def operation(list_):
        print(f"The mean is: {np.mean(list_)}")

class Max(Operations):
    """Compute max"""
    def operation(list_):
        print(f"The mean is: {np.max(list_)}")

class Main:
    """Main"""
    @abstractmethod
    def get_operations(list_):
        # __subclasses__ iterates over every class inheriting from Operations
        for operation in Operations.__subclasses__():
            operation.operation(list_)

if __name__ == "__main__":
    Main.get_operations([1, 2, 3, 4])

The mean is: 2.5
The mean is: 4


## THE LISKOV SUBSTITUTION PRINCIPLE (LSP)
### Functions that use pointer or refences to base classes must be able to use objects of derived classes without knowing it
Alternatively, this can be expressed as "Derived classes must be substituitable for their base classes".

In simpler words this means that if a subclass redefines a function also present in parent class, a client-user (remember the API part in the previous principle) shoud not be noticing any difference in **behaviour**, and it is a **substitute** for the base class.
For example, base class `Operations` in prev. example has a method `operation()` that does nothing.\
If a subclass overwrites the function `operation()` but this subclass actually returns a value instead of printing and passing, this would result in a different behaviour, not a catastrofic one but it changes how the function can be handled.

## THE INTERFACE SEGREGATION PRINCIPLE (ISP)
### Many client-specific interfaces are better than one general-purpose interface
In simpler words, and in a class-based context (Python) every class and its interface (every *exposed* property or method) should have only the interface needed & try to avoid exposing methods that not always are going to work.

In [10]:
# WRONG APPROACH: I inherited the class Mammals but its interface indicates that every mammal can 'walk'
# & 'swim' this is not correct if we consider whales as an example
import numpy as np
from abc import ABC, abstractmethod

class Mammals(ABC):
    @abstractmethod
    def swim() -> bool:
        print("Can swim")

    @abstractmethod
    def walk() -> bool:
        print("Can walk")

class Human(Mammals):
    def swim():
        return print("Humans can swim")

    def walk():
        return print("Humans can walk")

class Whale(Mammals):
    def swim():
        return print("Whales can swim")

# lets invoke the methods
Human.walk()
Human.swim()

Whale.walk() # <- whales cannot walk so why is this method accessed by whales?
Whale.swim()


Humans can walk
Humans can swim
Can walk
Whales can swim


In [13]:
# BEST APPROACH: the way ISP indicates is there should be more specific-client interfaces, lets refactor
# previous code having in mind this:
from abc import ABC, abstractmethod

class Walker(ABC):
    @abstractmethod
    def walk() -> bool:
        return print("Can walk")

class Swimmer(ABC):
    @abstractmethod
    def swim() -> bool:
        return print("Can swim")

class Human(Walker, Swimmer):
    def walk():
        return print("Humans can walk")

    def swim():
        return print("Humans can swim")

class Whale(Swimmer):
    def swim():
        return print("Whales can swim")

# lets invoke the methods
Human.walk()
Human.swim()

try:
    Whale.walk()
except AttributeError:
    print("Whales cannot walk")
Whale.swim()

Humans can walk
Humans can swim
Whales cannot walk
Whales can swim


## THE DEPENDENCY INVERSION PRINCIPLE (DIP)
### Abstractions should not depend on details. Details should depend on abstraction. High-level modules should not depend on low-level modules. Both should depend on abstractions

This basically indicates us that if we have an abstract class acting as a dependency of many modules, a change in that class would *break the code*, but under DIP we create a new interface for said abstract class, this interface would have to deal with the *adaptation* of each subclass or module

In [14]:
class FXConverter:
    def convert(self, from_currency, to_currency, amount):
        print(f'{amount} {from_currency} = {amount * 1.2} {to_currency}')
        return amount * 1.2

class App:
    def start(self):
        converter = FXConverter() # the app depends heavily on FXConverter() a change
        # in that class could change many behaviours if its being used for other modules.
        # lets say we don't wanna use the FXConverter but still want to convert some concurrencies
        converter.convert('EUR', 'USD', 100)

if __name__ == '__main__':
    app = App()
    app.start()

100 EUR = 120.0 USD


In [15]:
class CurrencyConverter(ABC):
    def convert(self, from_currency, to_currency, amount) -> float:
        pass

class FXConverter(CurrencyConverter):
    def convert(self, from_currency, to_currency, amount) -> float:
        print('Converting currency using FX API')
        print(f'{amount} {from_currency} = {amount * 1.2} {to_currency}')
        return amount * 2

class App:
    def __init__(self, converter: CurrencyConverter):
        self.converter = converter

    def start(self):
        self.converter.convert('EUR', 'USD', 100)

if __name__ == '__main__':
    converter = FXConverter() # in this case the class FXConverter is still being used but
    # it inherits from an abstract class that responds to a much higher need this is the inverse
    # way of thinking when writing OOP programs.
    app = App(converter)
    app.start()

Converting currency using FX API
100 EUR = 120.0 USD
